In [21]:
import os
import json
import time
import torch
import argparse
import numpy as np
from multiprocessing import cpu_count
from torch.utils.data import DataLoader
from collections import OrderedDict, defaultdict
import torch.nn as nn

from ptb import PTB
from utils import to_var, idx2word, expierment_name
from model import SentenceVAE

In [4]:
ts = time.strftime('%Y-%b-%d-%H-%M-%S', time.gmtime())

In [2]:
datasets = OrderedDict()

In [6]:
splits = ['train', 'valid'] + (['test'] if True else [])

In [7]:
splits

['train', 'valid', 'test']

In [9]:
for split in splits:
    datasets[split] = PTB(
        data_dir='data',
        split=split,
        create_data=True,
        max_sequence_length=60,
        min_occ=1
    )

Creating new TRAIN ptb data.
Vocablurary of 9877 keys created.
Creating new VALID ptb data.
Creating new TEST ptb data.


In [14]:
vocab_size = datasets['train'].vocab_size

In [16]:
sos_idx=datasets['train'].sos_idx

In [17]:
eos_idx=datasets['train'].eos_idx

In [18]:
pad_idx=datasets['train'].pad_idx

In [19]:
max_sequence_length = 60
embedding_size = 300
hidden_size = 256
word_dropout = 0.5
latent_size = 16
num_layers = 1
bidirectional = True

In [ ]:
class RVAE(nn.module):
    def __init__(self,ocab_size, embedding_size, max_sequence_length, hidden_size, word_dropout, latent_size,
                sos_idx, eos_idx, pad_idx, rnn_type='rnn' , num_layers=1, bidirectional=False):
        
        super().__init__()
        
        self.max_sequence_length = max_sequence_length
        self.sos_idx = sos_idx
        self.eos_idx = eos_idx
        self.pad_idx = pad_idx
        
        self.latent_size = latent_size
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        self.embedding_size = nn.Embedding(vocab_size,embedding_size)
        self.word_dropout = nn.Dropout(p=word_dropout)
        

            
        

In [25]:
class Encoder(nn.Module):
    def __init__(self,embedding_size, hidden_size, num_layers, bidirectional,latent_size,rnn_type):
        super(Encoder,self).__init__()
        
        self.latent_size = latent_size
        self.rnn_type = rnn_type
        self.bidirectional = bidirectional
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        
        if self.rnn_type == 'rnn':
            rnn = nn.RNN
        elif self.rnn_type == 'gru':
            rnn = nn.GRU
        elif self.rnn_type =='lstm':
            rnn = nn.LSTM
        else:
            raise ValueError()
        
        self.encoder = rnn(embedding_size,hidden_size,num_layers = num_layers,bidirectional = bidirectional)
        
        
        
        self.fc1 = nn.Linear(794, hidden_size_in)
        self.fc1_1 = nn.Linear(hidden_size_in, hidden_size)
        self.fc1_2 = nn.Linear(hidden_size_in, hidden_size)
        self.relu = nn.ReLU()
                        
    def encode(self,x):
        
        x = self.relu(self.fc1(x))
        mu = self.fc1_1(x)
        log_var = self.fc1_2(x)
                
        return mu,log_var
    
    def reparametrize(self, mu, logvar):
        std = logvar.mul(0.5).exp_()
        
        eps = torch.FloatTensor(std.size()).normal_()
        eps = Variable(eps).cuda()
        
        return eps.mul(std).add_(mu)
    
    def forward(self,x):
        mu, logvar = self.encode(x)
        reparam = self.reparametrize(mu,logvar)
        
        return mu,logvar,reparam
        
encoder = Encoder(5,5).cuda()